In [10]:
import numpy as np
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, value

On cherche à résoudre
\begin{gather*}
    \max_{\phi \in \mathcal{M}_{n,m}(\mathbb{R})} \sum_{i = 1}^{n} \sum_{j = 1}^{m} \phi_{i,j} V_{i,j} \\ \text{avec } \left\{
\begin{array}{l}
\forall i \in \{1,\dots n\}, \sum_{j = 1}^{m} \phi_{i,j} = \mu_i \\
\forall j, j' \in \{1, \dots, m\}, \sum_{i = 1}^{n} \phi_{i,j} \left( U_{i,j} - U_{i,{j'}} \right) \geqslant 0 \\
\forall (i,j) \in \{1,\dots n\} \times \{1, \dots, m\}, \phi_{i,j} \geqslant 0
\end{array}
\right.
\end{gather*}

La fonction ci-dessous prend en argument les matrices $U$, $V$ et le vecteur $\mu$ et renvoie la matrice $\phi$ optimale ainsi que la valeur max.

In [11]:
# Function to solve the linear programming problem
def solve_lp(U, V, mu):
    n, m = U.shape  # Dimensions of the problem

    # Create the LP problem
    lp_problem = LpProblem("Linear_Programming_Problem", LpMaximize)

    # Decision variables: phi[i][j] for each pair (i, j)
    phi = [[LpVariable(f"phi_{i}_{j}", lowBound=0) for j in range(m)] for i in range(n)]

    # Objective function: Maximize sum(phi[i][j] * V[i][j])
    lp_problem += lpSum(phi[i][j] * V[i][j] for i in range(n) for j in range(m)), "Objective"

    # Constraint 1: sum(phi[i][j]) = mu[i] for all i
    for i in range(n):
        lp_problem += lpSum(phi[i][j] for j in range(m)) == mu[i], f"mu_constraint_{i}"

    # Constraint 2: sum(phi[i][j] * U[i][j]) >= sum(phi[i][j] * U[i][j']) for all j, j'
    for j in range(m):
        for j_prime in range(m):
            if j != j_prime:
                lp_problem += (
                    lpSum(phi[i][j] * (U[i][j] - U[i][j_prime]) for i in range(n)) >= 0,
                    f"U_constraint_{j}_{j_prime}"
                )

    # Solve the problem
    lp_problem.solve()

    # Extract the solution
    solution = np.array([[phi[i][j].varValue for j in range(m)] for i in range(n)])
    
    max_value = value(lp_problem.objective)

    return solution, max_value

**Comportement du Sender en l'absence de signal :**

Sans signal, le Receiver choisit l'action qui maximise l'espérance selon son a priori. Si plusieurs actions la maximisent, il choisit de manière uniforme parmi celles-ci. Les fonctions ci-dessous calculent les utilités espérées initiales (sans signal) du Receiver et du Sender en fonction des utilités $U$ et $V$ et de leur a priori $\mu$.

In [12]:
def expected_utilities(U, V, mu):
    # Vérifier que les dimensions de U, V et mu correspondent
    n, m = U.shape
    if V.shape != U.shape or len(mu) != n:
        raise ValueError("Les dimensions de U, V et mu doivent correspondre.")
    
    # Calculer le produit scalaire de mu avec chaque colonne de U pour le Receiver
    dot_products_U = [np.dot(U[:, j], mu) for j in range(m)]
    max_utility_receiver = max(dot_products_U)
    
    # Trouver les indices des colonnes de U qui maximisent l'utilité pour le Receiver
    optimal_actions = [j for j, value in enumerate(dot_products_U) if value == max_utility_receiver]
    
    # Calculer le vecteur moyen pour les colonnes optimales de V pour le Sender
    mean_vector_V = np.mean(V[:, optimal_actions], axis=1)
    expected_utility_sender = np.dot(mean_vector_V, mu)
    
    return max_utility_receiver, expected_utility_sender

Application dans le cas du juge (Receiver) et du procureur (Sender) :

- **$S = \{s_1, s_2\}$** où $s_1 = \text{innocent}$ et $s_2 = \text{coupable}$
- **$A = \{a_1, a_2\}$** où $a_1 = \text{acquitter}$ et $a_2 = \text{condamner}$
- **$U = \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix}$**
- **$V = \begin{pmatrix} 0 & 1 \\ 0 & 1 \end{pmatrix}$**
- **$\mu = \begin{pmatrix} \alpha \\ 1 - \alpha \end{pmatrix}$**

In [15]:
alpha = 0.7

U = np.array([[1, 0],
              [0, 1]])

V = np.array([[0, 1],
              [0, 1]])

mu = np.array([alpha,
               1 - alpha])

phi, max_value = solve_lp(U, V, mu)

utilitee_esperee_initiale_receiver, utilitee_esperee_initiale_sender = expected_utilities(U, V, mu)

utilite_esperee_receiver = np.sum(np.array([phi[i,j]*U[i,j] for i in range(U.shape[0]) for j in range (U.shape[1])]))

# Solve the problem


# Display the solution
print("Phi optimal:")
print(phi)
print(f"Utilité espérée du procureur : {max_value}")
print(f"Utilité espérée initiale du procureur : {utilitee_esperee_initiale_sender}")


print(f"Utilité espérée du juge : {utilite_esperee_receiver}")
print(f"Utilité espérée initiale du juge : {utilitee_esperee_initiale_receiver}")

Phi optimal:
[[0.4 0.3]
 [0.  0.3]]
Utilité espérée du procureur : 0.6
Utilité espérée initiale du procureur : 0.0
Utilité espérée du juge : 0.7
Utilité espérée initiale du juge : 0.7


Maintenant, on veut résoudre
\begin{gather*}
                    \max_{\pi \in \mathcal{M}_{n,m}(\mathbb{R})} \sum_{\substack{1 \leqslant i \leqslant n \\ 1 \leqslant j, k \leqslant m}} \theta_{k,j}(\pi) \pi_{i,k} \mu_i V_{i,j}\\
                \text{avec }
                \left\{
                \begin{array}{l}
                \forall i \in \{1,\dots n\}, \sum_{k = 1}^{m} \pi_{i,k} = 1 \\
                \forall (i,k) \in \{1,\dots, n\} \times \{1, \dots, m\}, \pi_{i,k} \geqslant 0
                \end{array}
                \right. \\
                \text{où } \theta_{k,j}(\pi) = \frac{q_j\exp\left( \frac{g_{k,j}}{\varepsilon} \right)}{\sum_{j' = 1}^{m} q_{j'} \exp\left( \frac{g_{k,j'}}{\varepsilon} \right)} \text{ et } g_{k,j} = \sum_{i = 1}^n \frac{\pi_{i,k}\nu_i}{\sum_{i' = 1}^{n}\pi_{i',k}\nu_{i'}} U_{i,j}
\end{gather*}